# 0. EDA

# 0.1 Count unique values in a vector (while dropping the NaN)

In [ ]:
# This return a series
df['Borough'].value_counts(dropna=False)

In [ ]:
df.describe()

### Plots

In [ ]:
# Rescaling the axis using log
df['Existing Zoning Sqft'].plot(kind='hist', rot=70, logx=True, logy=True)
df.plot(kind='scatter', x='initial_cost', y='total_est_fee', rot=70)
df.boxplot(column='initial_cost', by='Borough', rot=90)

# 2.1 Data Cleaning : Melt, pivot, pivottable

# 2.1.1 Transform into panel data: pd.melt

In [ ]:
# df =        
#         Date  Ozone  Solar.R  Wind  Temp
# 0 1973-05-01   41.0    190.0   7.4    67
# 1 1973-05-02   36.0    118.0   8.0    72
# 2 1973-05-03   12.0    149.0  12.6    74
# 3 1973-05-04   18.0    313.0  11.5    62
# 4 1973-05-05    NaN      NaN  14.3    56

# Melt them all
pd.melt(frame=df, id_vars=['Date'])

#           Date variable  value
# 0   1973-05-01    Ozone   41.0
# 1   1973-05-02    Ozone   36.0
# 2   1973-05-03    Ozone   12.0
# 3   1973-05-04    Ozone   18.0
# 4   1973-05-05    Ozone    NaN
# 5   1973-05-06    Solar.R   28.0
# 6   1973-05-07    Solar.R   23.0

# Only keep the value of selected columns and name the new columns
pd.melt(frame=df, id_vars=['Date'], value_vars=['Temp', 'Wind'], var_name='Temp and Wind', value_name='Quantity')

#     Date Temp and Wind  Quantity
# 0     NaN          Temp      67.0
# 1     NaN          Temp      72.0
# 2     NaN          Temp      74.0
# 3     NaN          Temp      62.0
# 4     NaN          Temp      56.0
# 5     NaN          Temp      66.0
# 6     NaN          Temp      65.0b


### • Transform back: pd.pivot

In [ ]:
pd.pivot(data=df, index='Date', columns='variable', values='value')

# 2.1.2 Only Transform back part of columns and use function to the rest columns: pd.pivot_table
- Appliable when we only care about some of columns

In [ ]:
# df
#      A    B      C  D  E
# 0  foo  one  small  1  2
# 1  foo  one  large  2  4
# 2  foo  one  large  2  5
# 3  foo  two  small  3  5
# 4  foo  two  small  3  6
# 5  bar  one  large  4  6
# 6  bar  one  small  5  8
# 7  bar  two  small  6  9
# 8  bar  two  large  7  9

# In this example, we take the mean of bar-large-one and bar-large-tow and we get 5.50
# we use dictionary to manipulate different columns

table = pd.pivot_table(df, values=['D', 'E'], index=['A', 'C'],
                    aggfunc={'D': np.mean,
                             'E': [min, max, np.mean]})
# table
#                 D    E
#             mean  max      mean  min
# A   C
# bar large  5.500000  9.0  7.500000  6.0
#     small  5.500000  9.0  8.500000  8.0
# foo large  2.000000  5.0  4.500000  4.0
#     small  2.333333  6.0  4.333333  2.0

# 2.2 Data cleaning: Splitting columns using .str

# 2.2.1 Slicing the column name directly

In [ ]:
#      country  year variable   value
# 0         AD  2000     m014     0.0
# 1         AE  2000     m014     2.0
# 2         AF  2000     m014    52.0
# 3         AG  2000     m014     0.0
# 4         AL  2000     m014     2.0

# Create the 'gender' column
tb_melt['gender'] = tb_melt.variable.str[0]b

# Create the 'age_group' column
tb_melt['age_group'] = tb_melt.variable.str[1:]

# 2.2.2 Use certain pattern to split them and get splitted parts

In [ ]:
# Create the 'str_split' column
ebola_melt['str_split'] = ebola_melt['type_country'].str.split('_')

# In [4]: ebola_melt.head()
# Out[4]: 
#          Date  Day  type_country  counts        str_split
# 0    1/5/2015  289  Cases_Guinea  2776.0  [Cases, Guinea]
# 1    1/4/2015  288  Cases_Guinea  2775.0  [Cases, Guinea]
# 2    1/3/2015  287  Cases_Guinea  2769.0  [Cases, Guinea]
# 3    1/2/2015  286  Cases_Guinea     NaN  [Cases, Guinea]
# 4  12/31/2014  284  Cases_Guinea  2730.0  [Cases, Guinea]

# Create the 'type' column
ebola_melt['type'] = ebola_melt['str_split'].str.get(0)

# Create the 'country' column
ebola_melt['country'] = ebola_melt['str_split'].str.get(1)

# 2.2.3 Use expand=True directly but need to rename the columns

In [ ]:
ebola_melt = ebola_melt.drop(columns = ['type_country']).concat(ebola_melt['type_country'].str.split('_', expand=True))

# 2.3 Data cleaning: Concat, Merge and Join data

    - Concat gives the flexibility to join based on the axis( all rows or all columns)
    - Append is the specific case(axis=0, join='outer') of concat
    - Join is based on the indexes (set by set_index) on how variable =\['left','right','inner','couter'\]
    - Merge is based on any particular column each of the two dataframes, this columns are variables on like 'left_on', 'right_on', 'on'

In [ ]:
# Default is to concat dfs row-wise. they should have the same columns
row_concat = pd.concat([uber1, uber2, uber3])

In [ ]:
# Concat dfs column-wise. they should have the same number of rows.
ebola_tidy = pd.concat([ebola_melt, status_country], axis=1)

In [ ]:
m2m = pd.merge(left=m2m, right=survey, left_on='ident', right_on='taken')

# 2.4 Data cleaning: using regular expressions

# 2.4.5 Check if the pattern is matched

In [5]:
''' Compile pattern then match'''

# Import the regular expression module
import re

# Compile the pattern: prog
prog = re.compile('\d{3}-\d{3}-\d{3}')

# See if the pattern matches
result = prog.match('123-456-7890')
print(bool(result))

''' Match directly'''
pattern1 = bool(re.match(pattern='\d{3}-\d{3}-\d{4}', string='123-456-7890'))
pattern2 = bool(re.match('\$\d*\.\d{2}', string = '$123.45'))
pattern3 = bool(re.match('[A-Z]\w*', string = 'Australia'))




True


# 2.4.6 Modify columns using certain pattern - use re.findall and .replace


In [ ]:
matches = re.findall('\d+', 'the recipe calls for 10 strawberries and 1 banana')
# for dfs The data is like $16.99 
tips['total_dollar_re'] = tips.total_dollar.apply(lambda x: re.findall('\d+\.\d+', x)[0])
# use replace to modify values
tips['total_dollar_replace'] = tips.total_dollar.apply(lambda x: x.replace('$', ''))

# 2.5 Deal with missing values and dupilcate values

In [ ]:
'''Drop duplicate values: df.drop_duplicates()'''
tracks_no_duplicates = tracks.drop_duplicates()

'''Drop NA''' 
df.dropna(how = 'any', axix=1) # if any missing value, drop column
df.dropna(how = 'all', axis=0) # if all missing value, drop row

# check the sum of missing values
df.isnull().sum()

'''Replace NaN with something'''
oz_mean = airquality.Ozone.mean()

# Replace all the missing values in the Ozone column with the mean
airquality['Ozone'] = airquality.Ozone.fillna(airquality.Ozone.mean())




# 1.Data manipulation

## 1.1 dict manipulation

###  1.1.1 Extract / Modify indexes

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
os.getcwd()

In [ ]:
os.chdir('C:\\Users\\Dell\\Desktop\\MachineLearning\\Notebooks')

In [ ]:
europe = {'spain':'madrid', 'france':'paris', 'germany':'berlin', 'norway':'oslo' }

# Add italy to europe
europe['italy']='rome'

# 1.2 Dataframe Manipulation

don't forget .reset_index() everytime after merging them

# 1.2.1 Converting everything into dataframe

In [ ]:
# numpy array : turnout_zscore is an n*1 array
election['turnout_zscore'] = turnout_zscore # OR
pd.Dataframe(turnout_zscore)


# 1.2.2 Change data types

    You can use this to manipulating datetime
    You can also use it to create dummy varibles for columns that already have binary values
    Use error=coerce whenever necessary

In [ ]:
# datetime and numeric type 
# Corerce in case there are some dirty values
News.date = pd.to_datetime(News.date)
pd.to_numeric(News.year, errors='coerce')
ser.astype('int64')

# extract years from datetime
df['year'] = pd.DatetimeIndex(df['birth_date']).year



# 1.2.3 extract / modify indexes

In [ ]:
row_labels = ['US', 'AUS', 'JPN', 'IN', 'RU', 'MOR', 'EG']

cars.index = row_labels

# 1.2.4 Read CSV

In [ ]:
cars = pd.read_csv('cars.csv', index_col=0)
cars = pd.read_csv('cars.csv', index_col='serial number', parse_dates='Date')

# 1.2.5 Read multiple files with a similar pattern 

In [ ]:
# read all csv files

# Import necessary modules
import pandas as pd
import glob

# Write the pattern: pattern.  * is a wild card
pattern = '*.csv'

# Save all file matches: csv_files
'''csv_files is a list full of file names'''
csv_files = glob.glob(pattern)

# Load the second file into a DataFrame: csv2
csv1 = pd.read_csv(csv_files[0])

# Load all files and put them into a list
frames = []
for csv in csv_files:
    df = pd.read_csv(csv)
    frames.append(df)


# 1.2.5.1 Read all csv files into a list and concat them all

In [ ]:
# Create an empty list: frames
frames = []

#  Iterate over csv_files
for csv in csv_files:

    #  Read csv into a DataFrame: df
    df = pd.read_csv(csv)
    
    # Append df to frames
    frames.append(df)

# Concatenate frames into a single DataFrame: uber
uber = pd.concat(frames)

# 1.2.5.2 A home-made csv-reading into a dictionary

In [ ]:
fx_dict = {}
news_dict = {}
for i in range(1, 16):
    if i < 10:
        fx_dict['20190{}'.format(i)] = pd.read_csv('fx_20190{}.csv'.format(i), parse_dates=['time'])
        news_dict['20190{}'.format(i)] = pd.read_csv('News_20190{}.csv'.format(i), parse_dates=['date'])
    elif i < 13: 
        fx_dict['2019{}'.format(i)] = pd.read_csv('fx_2019{}.csv'.format(i), parse_dates=['time'])
        news_dict['2019{}'.format(i)] = pd.read_csv('News_2019{}.csv'.format(i), parse_dates=['date'])
    else: 
        fx_dict['20200{}'.format(i-12)] = pd.read_csv('fx_20200{}.csv'.format(i-12), parse_dates=['time'])
        news_dict['20200{}'.format(i-12)] = pd.read_csv('News_20200{}.csv'.format(i-12), parse_dates=['date'])

# 1.2.5.3 Export csv files all altogether

In [ ]:
for i in range(1, 16):
    if i < 10:
        fx[(fx.time.dt.year == 2019) & (fx.time.dt.month == i)].to_csv('fx_20190{}.csv'.format(i))
        News[(News.date.dt.year == 2019) & (News.date.dt.month == i)].to_csv('News_20190{}.csv'.format(i))
    elif i < 13:
        fx[(fx.time.dt.year == 2019) & (fx.time.dt.month == i)].to_csv('fx_2019{}.csv'.format(i))
        News[(News.date.dt.year == 2019) & (News.date.dt.month == i)].to_csv('News_2019{}.csv'.format(i))
    else:
        fx[(fx.time.dt.year == 2020) & (fx.time.dt.month == i-12)].to_csv('fx_20200{}.csv'.format(i-12))
        News[(News.date.dt.year == 2020) & (News.date.dt.month == i-12)].to_csv('News_20200{}.csv'.format(i-12))

### Extract elements / slicing pieces from df

In [ ]:
# extract: double [] returns df, single[] returns series
cars.loc[:,['country','drives_right']]
cars.loc[['RU', 'CN'],['country','drives_right']]
cars.iloc[[1, 2, 3],['country','drives_right']]
cars.iloc[[1, 2, 3],[1, 2, 3]]

# slicing: single[] returns df.
p_counties = election.loc['Perry': 'Potter', :]
p_counties = election.loc[: 'Potter', :]
p_counties_rev = election.loc['Potter': 'Perry': -1] # slice and arrange them reversely



### slicing with condition - including Non-zero or Non-null condition

In [ ]:
# slicing with condition: single[]
high_turnout = election.turnout > 70
high_turnout_df = election[high_turnout]

# Multiple conditions
df[(df.salt >= 50) & (df.eggs < 200)]

df2.loc[:, df2.all()] # keep columns with all non-zero values
df2.loc[:, df2.any()] # keep columns with not-all-zero values
df.loc[:, df.isnull().any()] # keep columns with any NaN values
df.loc[:, df.notnull().all()] # keep columns with no NaN values

# create columns using loc

# change all elements in a column - including creating dummies
    - Using pd.series.apply() or .map()
    - Using vectorized functions if the we want shorter running time - at the same speed with compiled code(C)
    - If the column already has clean binary values (like 'Male' and 'Female'), use .astype('category') directly.

In [ ]:
# Without condition

for lab, row in cars.iterrows():
    cars.loc[lab, "COUNTRY"] = row["country"].upper()
#
cars["COUNTRY"] = cars["country"].apply(str.upper)

# Create dummy in the dummiest way

def recode_gender(gender):
    if gender == 'Female':
        return 0
    elif gender == 'Male':
        return 1 
    else:
        return np.nan
tips['recode'] = tips.sex.apply(recode_gender)

# with condition: perfect for creating dummy or categorical variables - using map
red_vs_blue = {'Obama':'blue', 'Romney':"red"}
election['color'] = election.winner.map(red_vs_blue)

# Use np.where - faster

merged['if_16_NaN'] = np.where(merged.specialty_description_16.isnull(), 1, 0)


# if else & numpy operator

In [ ]:
if room == "kit" :
    print("looking around in the kitchen.")
elif room == "bed":
    print("looking around in the bedroom.")
else :
    print("looking around elsewhere.")

In [ ]:
bmi = np.array([ 21.852, 20.975, 21.75 , 24.747, 21.441]) 

In [ ]:
bmi > 21

## 1.3 list manipulation

### concatenate and slicing

In [ ]:
    # Concatenate shout1 with shout2: new_shout
    new_shout = shout1 + shout2
    # extract elements
    even_nums = (2,nums[1],nums[2])

### apply a function to every element in a list - use map - can use list comprehension

In [ ]:
# Create a list of strings: spells
spells = ["protego", "accio", "expecto patronum", "legilimens"]

# Use map() to apply a lambda function over spells: shout_spells
shout_spells = map(lambda item: item + '!!!', spells)

# Convert shout_spells to a list: shout_spells_list
shout_spells_list = list(shout_spells)

# Print the result
print(shout_spells_list)

### filter every element in a list 
    - can use list comprehension instead
    - can be used to pandas series as well

In [ ]:
# use list comprehension instead
new_fellowship = [member for member in fellowship if len(member) >= 7]

# Create a list of strings: fellowship
fellowship = ['frodo', 'samwise', 'merry', 'pippin', 'aragorn', 'boromir', 'legolas', 'gimli', 'gandalf']

# Use filter() to apply a lambda function over fellowship: result
result = filter(lambda member: len(member) > 6, fellowship)

# Convert result to a list: result_list
result_list = list(result)

# Print result_list
print(result_list)

### add up every element of the list- reduce

In [ ]:
# Create a list of strings: fellowship
fellowship = ['frodo', 'samwise', 'merry', 'pippin', 'aragorn', 'boromir', 'legolas', 'gimli', 'gandalf']

# Use filter() to apply a lambda function over fellowship: result
result = filter(lambda member: len(member) > 6, fellowship)

# Convert result to a list: result_list
result_list = list(result)

# Print result_list
print(result_list)

### count the items in one df and put them in to dict

In [ ]:
# Define count_entries()
def count_entries(df, col_name):
    """Return a dictionary with counts of 
    occurrences as value for each key."""

    # Initialize an empty dictionary: langs_count
    langs_count = {}
    
    # Extract column from DataFrame: col
    col = df[col_name]
    
    # Iterate over lang column in DataFrame
    for entry in col:

        # If the language is in langs_count, add 1
        if entry in langs_count.keys():
            langs_count[entry] += 1
        # Else add the language to langs_count, set the value to 1
        else:
            langs_count[entry] = 1

    # Return the langs_count dictionary
    return langs_count

# Call count_entries(): result
result = count_entries(tweets_df, 'lang')

# Print the result
print(result)

# loop

### while loop

In [ ]:
while offset != 0 :
    print("correcting...")
    offset = offset - 1
    print(offset)

### for loop

### for loop with enumerate

In [ ]:
areas = [11.25, 18.0, 20.0, 10.75, 9.50]
for index, area in enumerate(areas) :
    index = index + 1
    print("room " + str(index) + ": " + str(area))

### loop through dicts

In [ ]:
# Definition of dictionary
europe = {'spain':'madrid', 'france':'paris', 'germany':'berlin',
          'norway':'oslo', 'italy':'rome', 'poland':'warsaw', 'austria':'vienna' }
          
# Iterate over europe
for key, value in europe.items():
    print("the capital of "+ str(key)+" is "+ str(value))

### loop over numpy

In [ ]:
# For loop over np_height
for x in np_height:
    print(str(x) + " inches")

# For loop over every element in 2d array np_baseball
for x in np.nditer(np_baseball):
    print(str(x))

In [ ]:
# Iterate over rows of cars
# This prints every index name and all contents in that row.
for lab, row in cars.iterrows():
    print(lab)
    print(row)


# rand

In [ ]:
np.random.seed(123)

# Generate and print random float
print(np.random.rand())
print(np.random.randint(1,7))

### rand walk

In [ ]:
# numpy and matplotlib imported, seed set

# Simulate random walk 250 times
all_walks = []
for i in range(10) :
    random_walk = [0]
    for x in range(100) :
        step = random_walk[-1]
        dice = np.random.randint(1,7)
        if dice <= 2:
            step = max(0, step - 1)
        elif dice <= 5:
            step = step + 1
        else:
            step = step + np.random.randint(1,7)

        # Implement clumsiness
        if ___ :
            step = 0

        random_walk.append(step)
    all_walks.append(random_walk)

# Create and plot np_aw_t
np_aw_t = np.transpose(np.array(all_walks))
plt.plot(np_aw_t)
plt.show()

# try-except error handling- datacamp toolbox 1

# Other notes

In [ ]:
'''Why use set for stopwords?
'''
# Lists are slightly faster than sets when you just want to iterate over the values. Sets, however, are significantly faster than lists if you want to check if an item is contained within it. 

# Escape special characters when explicitly using regular expressions.
    use / before the re

In [ ]:
s = pd.Series(["1+1=2"])
s.str.split(r"\+|=", expand=True)
     0    1    2
0    1    1    2

# Assert

In [ ]:
# If the value is true, then it will not return anything. If False, it returns an AssertionError
assert ebola.notnull().all().all()

# Timeit

In [ ]:
import timeit
input_list = range(100)
# def div_by_five(num):
#     if num % 5 == 0:
#         return True
#     else:
#         return False
# xyz = (i for i in input_list if div_by_five(i))


print(timeit.timeit('''def div_by_five(num):
    if num % 5 == 0:
        return True
    else:
        return False
xyz = (i for i in input_list if div_by_five(i))
''', number = 5000))


# Useful shortcuts - vscode

In [ ]:
# ctrl + k + c : turn snippet into comments
# ctrl + k + u : turn comments back into codes